# Canadian Tide Time Series Data Downloading

**1. Set up Enviroment** italicized text

In [ ]:
!pip install requests
!pip install pandas
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

**2. User Input**

The script will ask for the follow:

- Station number
- Time Series Resolution
- Start Date
- End Date
- Tide Series

In [ ]:
stn_num = int(input('Please Provide Station Number: '))
data_res = input('Provide number of Resolution for Data Download 1, 3, 15, 60: ')
start_date = input('Provide Start Date in YYYY-MM-DD Format: ')
end_date = input('Provide End Date in YYYY-MM-DD Format: ')
tide_series = input('Provide Tide Series Code wlo, wlp: ')

Please Provide Station Number: 0491
Provide number of Resolution for Data Download 1, 3, 15, 60: 1
Provide Start Date in YYYY-MM-DD Format: 2025-05-28
Provide End Date in YYYY-MM-DD Format: 2025-05-29
Provide Tide Series Code wlo, wlp: wlo


**2. Find station endpoint based on station number provided**

In [ ]:
# Correcting to have Station in 5 digit format
stn_num = f"{stn_num:05d}"
stn_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={stn_num}'
stn_url

'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code=00491'

In [ ]:
df = pd.read_json(stn_url)
stn_id = df['id'].to_list()
stn_id

['5cebf1e23d0f4a073c4bbfac']

**3. Based on input resolution prgram will select the correct time series resolution**

In [ ]:
if data_res == '1':
    resolution = 'ONE_MINUTE'
elif data_res == '3':
    resolution = 'THREE_MINUTES'
elif data_res == '15':
    resolution = 'FITEEN_MINUTES'
else:
    resolution = 'SIXTY_MINUTES'

4. Create incremental list of Min Dates until more or greater than the Max Date.

In [ ]:
format = '%Y-%m-%d'
ma = datetime.strptime(end_date, format).date()
mi = datetime.strptime(start_date, format).date()
dayoffset = 6
start_dates =[]
end_dates = []
while mi < ma:
    start_dates.append(mi.strftime(format))
    mi = mi + timedelta(days=dayoffset)
    end_dates.append(mi.strftime(format))
end_dates[-1] = end_date
print(start_dates)
print(end_dates)



['2025-05-28']
['2025-05-29']


5. Create a list urls to request Tidal data from the API grouped by 7 day intervalsrequest until end date limit reached

In [ ]:
request_list = []

i = 0
for i in range(len(start_dates)):
  sd = start_dates[i]
  ed = end_dates[i]
  data_url = (f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/' +
                f'stations/5cebf1e33d0f4a073c4bc21f/data?time-series-code={tide_series}&from=' +
                f'{sd}T00%3A00%3A00Z&to={ed}T23%3A59%3A59Z&'+
                f'resolution={resolution}')
  request_list.append(data_url)


In [ ]:
request_list

['https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5cebf1e33d0f4a073c4bc21f/data?time-series-code=wlo&from=2025-05-28T00%3A00%3A00Z&to=2025-05-29T23%3A59%3A59Z&resolution=ONE_MINUTE']

7. Append data to a new Dataframe and export as a csv

In [ ]:
data = pd.DataFrame()
for url in request_list:
  df = pd.read_json(url)
  data = pd.concat([data, df])
  time.sleep(1)
data

data.to_csv(f'{stn_num}_{start_date}_{end_date}_{resolution}.csv')

8. Convert the CSV into Caris Tide File format

In [ ]:
data['date_time'] = pd.to_datetime(data['eventDate'])
data['date'] = data['date_time'].dt.date
data['time'] = data['date_time'].dt.time
data = data.drop(columns=['eventDate'])

tide_data = data[['date', 'time', 'value']]
fname = f'{stn_num}_{start_date}_{end_date}_{resolution}'

with open(f'{fname}_CarisTide.txt', 'w') as f:
    dfAsString = tide_data.to_string(header=False, index=False)
    f.write(dfAsString)
    f.close()

with open(f'{fname}_CarisTide.tid', 'w') as f2:
    f2.write('--------\n')

    file = open(f'{fname}_CarisTide.txt', 'r')
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line=='' or line=="/n":
            pass
        else:
            f2.write(line + '\n')
    file.close()


NameError: name 'remove' is not defined